# Podcast Searcher for my favorite podcasts

## Podcasts Available:
* Joe Rogan
* Chris Williamson
* The Diary Of A CEO


## The python code provided here is for Data Collection

In [ ]:
# Accessing the Youtube Data API
!pip install google-api-python-client

In [3]:
# Importing the necessary libraries
from googleapiclient.discovery import build
import pandas as pd

In [4]:
# Channel ids for Joe Rogan Podcast, Chris Williamson, A diary of CEO
channel_ids = ['UCzQUP1qoWDoEbmsQxvdjxgQ', 'UCIaH-gZIVC432YRjNVvnyCA', 'UCGq-a57w-aPwyi3pW7XLiHw']

In [ ]:
# Neccessary credentials
api_service_name = "youtube"
api_version = "v3"
api_key="ENTER YOUR API KEY -> GET IT FROM GOOGLE DEVELOPERS CONSOLE"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_ids)    #Get the details of all three channels
)
response = request.execute()

response

In [6]:
def get_channel_stats(youtube, channel_ids):
  all_data = []

  req = youtube.channels().list(
      part="snippet, contentDetails,statistics",
      id=','.join(channel_ids)
  )

  res = req.execute()

  for item in response['items']:
    data = {'Channel': item['snippet']['title'],
            'Subscribers': item['statistics']['subscriberCount'],
            'Views' : item['statistics']['viewCount'],
            'Videos': item['statistics']['videoCount'],
            'PlayListID': item['contentDetails']['relatedPlaylists']['uploads']
            }

    all_data.append(data)

  return (pd.DataFrame(all_data))


In [ ]:
get_channel_stats(youtube, channel_ids)

In [9]:
# Consisting of all the videos of the three respective channels
playlist_ids = ['UUzQUP1qoWDoEbmsQxvdjxgQ', 'UUIaH-gZIVC432YRjNVvnyCA', 'UUGq-a57w-aPwyi3pW7XLiHw']

In [29]:
# Pick an id one by one
playlist_id = "UUGq-a57w-aPwyi3pW7XLiHw"

In [30]:
def get_video_ids(youtube, playlist_ids):

  video_ids = []

  req = youtube.playlistItems().list(
      part="snippet, contentDetails",
      playlistId = playlist_id,
      maxResults = 50
  )
  res = req.execute()

  for item in res['items']:
    video_ids.append(item['contentDetails']['videoId'])

  next_page_token = res.get('nextPageToken')
  while next_page_token is not None:
    req = youtube.playlistItems().list(
      part="snippet, contentDetails",
      playlistId = playlist_id,
      maxResults = 50,
      pageToken = next_page_token   # To get all the ids
  )
    res = req.execute()

    for item in res['items']:
      video_ids.append(item['contentDetails']['videoId'])

    next_page_token = res.get('nextPageToken')

  return video_ids


In [31]:
video_ids = get_video_ids(youtube, playlist_ids)
len(video_ids)

299

In [ ]:
video_ids

In [ ]:
ceo_df = pd.DataFrame(video_ids, columns=['id'])
ceo_df.head()

In [33]:
# Convert the ids to csv
ceo_df.to_csv('diary_of_a_ceo.csv', index=False)